In [ ]:
!pip install PyPortfolioOpt
!pip install yfinance
!pip install forex-python

In [1]:
import pandas as pd
import yfinance as yf
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import BlackLittermanModel

In [4]:
# Read in price data
df = stock_prices

# Calculate expected returns and sample covariance
mu = expected_returns.capm_return(df)
S = risk_models.CovarianceShrinkage(df).ledoit_wolf()

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S) #Add constraints: ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.1)) #Max 10% on each weight.
ef.add_objective(objective_functions.L2_reg, gamma=1) #Reduce zero-weights by adding penalty on small weights with a L2-regularization term
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights_capm_LedoitWolf.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict({'BG.VI': 0.17198, 'CABK.MC': 0.16382, 'DANSKE.CO': 0.16595, 'FRO.OL': 0.26551, 'MCD': 0.05812, 'NDA-FI.HE': 0.17463})
Expected annual return: 8.9%
Annual volatility: 25.3%
Sharpe Ratio: 0.27


/usr/local/python/3.12.1/lib/python3.12/site-packages/pypfopt/expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
/usr/local/python/3.12.1/lib/python3.12/site-packages/pypfopt/expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
/usr/local/python/3.12.1/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


(0.08939934308264025, 0.25256952708958724, 0.2747732233668239)

In [26]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


latest_prices = get_latest_prices(df)

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'MCD': 14, 'BG.VI': 51, 'CABK.MC': 203, 'NDA-FI.HE': 89, 'DANSKE.CO': 1}
Funds remaining: $126.89


### Black-Litterman allocation

In [14]:
S = risk_models.sample_cov(df)
viewdict = {"AAPL": 0.20, "BBY": -0.30, "BAC": 0, "SBUX": -0.2, "T": 0.131321}
bl = BlackLittermanModel(S, pi="equal", absolute_views=viewdict, omega="default")
rets = bl.bl_returns()

ef = EfficientFrontier(rets, S)
ef.max_sharpe()

OrderedDict([('GOOG', 0.0),
             ('AAPL', 0.2516700243034513),
             ('FB', 0.0061410740896274),
             ('BABA', 0.0504178900290811),
             ('AMZN', 0.0),
             ('GE', 0.0),
             ('AMD', 0.0),
             ('WMT', 0.0),
             ('BAC', 0.0),
             ('GM', 0.0),
             ('T', 0.6703436046652544),
             ('UAA', 0.0),
             ('SHLD', 0.0),
             ('XOM', 0.0214274069125859),
             ('RRC', 0.0),
             ('BBY', 0.0),
             ('MA', 0.0),
             ('PFE', 0.0),
             ('JPM', 0.0),
             ('SBUX', 0.0)])


- Frontline (FRO.OL): NOK (listed on the Oslo Stock Exchange)
- Nordea Bank Abp (NDA-FI.HE): EUR (listed on the Helsinki Stock Exchange)
- BAWAG Group AG (BG.VI): EUR (listed on the Vienna Stock Exchange)
- CaixaBank (CABK.MC): EUR (listed on the Madrid Stock Exchange)
- Danske Bank (DANSKE.CO): DKK (listed on the Copenhagen Stock Exchange)
- McDonald’s (MCD): USD (listed on the NYSE)



In [2]:


# List of ticker symbols
tickers = ['FRO.OL', 'NDA-FI.HE', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'MCD']

# Download historical data for the specified tickers
stock_data = yf.download(tickers, start='2008-01-01', end='2024-09-10')

# Display the first few rows of the data
display(stock_data.head())






[*********************100%***********************]  6 of 6 completed


Price                     Adj Close                                     \
Ticker                        BG.VI   CABK.MC   DANSKE.CO       FRO.OL   
Date                                                                     
2008-01-02 00:00:00+00:00       NaN  2.082304  122.023239  1174.019775   
2008-01-03 00:00:00+00:00       NaN  2.042259  120.215508  1180.702759   
2008-01-04 00:00:00+00:00       NaN  2.042259  117.955780  1122.781738   
2008-01-07 00:00:00+00:00       NaN  1.982193  119.010307  1060.404907   
2008-01-08 00:00:00+00:00       NaN  2.002215  120.215508  1075.999146   

Price                                          Close                      \
Ticker                           MCD NDA-FI.HE BG.VI CABK.MC   DANSKE.CO   
Date                                                                       
2008-01-02 00:00:00+00:00  36.280479  4.380477   NaN    5.20  187.387222   
2008-01-03 00:00:00+00:00  36.174328  4.392222   NaN    5.10  184.611115   
2008-01-04 00:00:00+00:00  35.624817  4.306100   NaN    5.10  181.140976   
2008-01-07 00:00:00+00:00  36.236767  4.321758   NaN    4.95  182.760376   
2008-01-08 00:00:00+00:00  35.643562  4.321758   NaN    5.00  184.611115   

Price                              ...        Open                     \
Ticker                     FRO.OL  ...   DANSKE.CO  FRO.OL        MCD   
Date                               ...                                  
2008-01-02 00:00:00+00:00  1317.5  ...  184.842453  1302.5  59.480000   
2008-01-03 00:00:00+00:00  1325.0  ...  186.461853  1315.0  58.400002   
2008-01-04 00:00:00+00:00  1260.0  ...  184.611115  1317.5  57.330002   
2008-01-07 00:00:00+00:00  1190.0  ...  180.909637  1247.5  57.360001   
2008-01-08 00:00:00+00:00  1207.5  ...  184.148422  1160.0  58.439999   

Price                               Volume                                  \
Ticker                    NDA-FI.HE  BG.VI    CABK.MC  DANSKE.CO    FRO.OL   
Date                                                                         
2008-01-02 00:00:00+00:00     11.36    NaN  3370267.0   239259.0  131921.0   
2008-01-03 00:00:00+00:00     11.18    NaN  2046484.0   427514.0  145772.0   
2008-01-04 00:00:00+00:00     11.20    NaN  3184218.0   396353.0  244393.0   
2008-01-07 00:00:00+00:00     11.00    NaN  4010529.0   446996.0  248477.0   
2008-01-08 00:00:00+00:00     11.11    NaN  7756402.0  2476565.0  260464.0   

Price                                             
Ticker                            MCD  NDA-FI.HE  
Date                                              
2008-01-02 00:00:00+00:00   7858300.0   550381.0  
2008-01-03 00:00:00+00:00   6514700.0   748733.0  
2008-01-04 00:00:00+00:00   9687500.0  2332216.0  
2008-01-07 00:00:00+00:00  10784500.0  2407069.0  
2008-01-08 00:00:00+00:00  10080800.0   451652.0  

[5 rows x 36 columns]

In [3]:
# Extract the adjusted close prices
stock_prices = stock_data['Adj Close'].reset_index()
stock_prices.columns.name = None
stock_prices.set_index('Date', drop = True, inplace = True)
display(stock_prices.head())

,BG.VI,CABK.MC,DANSKE.CO,FRO.OL,MCD,NDA-FI.HE
Date,,,,,,
2008-01-02 00:00:00+00:00,NaN,2.082304,122.023239,1174.019775,36.280479,4.380477
2008-01-03 00:00:00+00:00,NaN,2.042259,120.215508,1180.702759,36.174328,4.392222
2008-01-04 00:00:00+00:00,NaN,2.042259,117.955780,1122.781738,35.624817,4.306100
2008-01-07 00:00:00+00:00,NaN,1.982193,119.010307,1060.404907,36.236767,4.321758
2008-01-08 00:00:00+00:00,NaN,2.002215,120.215508,1075.999146,35.643562,4.321758
